In [16]:
import os
import os.path

import pickle
import pandas as pd

from tqdm import tqdm

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import matplotlib.pyplot as plt

# 1. Загрузим ИНН и названия вузов

In [4]:
with open('..\\data\\interim\\vuz_inn.pickle', 'rb') as f:
        vuz_inn = pickle.load(f)

In [5]:
vuz_inn.head()

,region_name,region_code,full_name,post_address,inn,is_avtonomnaya,is_akkreditovannoye,is_byudzhetnoye,is_voyennoye,is_gosudarstvennoye,...,is_inclusive,is_kazennoye,is_kultury,is_mezhgosudarstvennoye,is_munitsipalnoye,is_negosudarstvennaya,is_nekommercheskaya,is_technologies,is_federalnoye,is_chastnoye
0,Тюменская область,72,федеральное государственное казенное военное о...,"625001, г. Тюмень, улица Льва Толстого, д. 1",7203255254,False,False,False,True,True,...,False,True,False,False,False,False,False,False,True,False
1,Вологодская область,35,федеральное государственное бюджетное образова...,"162600, Вологодская область, г. Череповец, про...",3528051834,False,False,True,False,True,...,False,False,False,False,False,False,False,False,True,False
2,г. Москва,77,федеральное государственное бюджетное образова...,"117997, г. Москва, Стремянный переулок, д. 36",7705043493,False,False,True,False,True,...,False,False,False,False,False,False,False,False,True,False
3,"образовательные учреждения, находящиеся за пре...",90,Государственное образовательное учреждение выс...,"Республика Армения, г. Ереван, ул. Овсепа Эмин...",00053474,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Воронежская область,36,Воронежский институт высоких технологий - авто...,"394043, г. Воронеж, ул. Ленина, д. 73а",3662006761,True,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


# 2. Посчитаем для каких вузов доступна финансовая информация

Источник данных - сведения о планах финансово-хозяйственной деятельности https://bus.gov.ru/open-data.
Определим, для каких вузов финансовая информация доступна на bus.gov.ru. 
Берем финансовую информацию за все года что есть.

In [15]:
# %%pycodestyle
data_dir = os.path.dirname(os.getcwd()) + '\\data\\interim'
file_list = os.listdir(data_dir)
if 'years_num.pickle' in file_list:
    print('нашлось сохраненное years_num')
    with open('..\\data\\interim\\years_num.pickle', 'rb') as f:
        years_num = pickle.load(f)
else:
    years_num = []
    for i in tqdm(range(800)):
        inn = vuz_inn['inn'][i]
        url = 'https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn
        response = requests.get(url)
        data = response.json()

        if data['content'] == []:
            years_num.append(0)
        else:
            temp = []
            for i in range(len(data['content'])):
                temp.append(data['content'][i]['common']['financialYear'])
            years_num.append(len(temp))
    with open('..\\data\\interim\\years_num.pickle', 'wb') as f:
        pickle.dump(years_num, f)

vuz_inn['years_num'] = years_num

years_num_unique = vuz_inn['years_num'].unique()

print('Финансовая информация доступна для',
      len(vuz_inn[vuz_inn['years_num'] > 0])*100/800,
      '% ВУЗов')

TOTAL_NUMBER = 800
FIN_INFO_AVAILABLE = len(vuz_inn[vuz_inn['years_num'] > 0])
FIN_INFO_UNAVAILABLE = len(vuz_inn[vuz_inn['years_num'] == 0])

vus_general = {'Финансовая информация доступна': FIN_INFO_AVAILABLE,
               'Финансовая информация НЕдоступна':FIN_INFO_UNAVAILABLE}
vus_general_ser = pd.Series(vus_general) 

# Make the plot with pandas
vus_general_ser.plot(kind='pie', subplots=True)
plt.title("Набор данных по вузам")
plt.ylabel("")
plt.show()

нашлось сохраненное years_num
Финансовая информация доступна для 71.625 % ВУЗов


NameError: name 'plt' is not defined